# ME 617 HW 3

**Author: Cameron Irmas**

_Note: This source code was implemented in Julia. This notebook requires a Julia 1.7.x kernel as well as an environment that includes the necessary dependencies. See the [repo](https://github.com/camirmas/DesignAutomation) for full implementations, testing, and dependency information. All relevant code has been copied into this notebook, so no importing of individual modules is necessary._

In [12]:
using LinearAlgebra
using Optim

In [13]:
function levy3(x)
    n = length(x)

    y(x_i) = 1 + (x_i-1)/4

    term1 = sin(π*y(x[1]))^2
    term3 = y(x[n]-1)^2 * (1+sin(2π*y(x[n]))^2)
    
    sum = 0
    for i=1:n-1
        new = (y(x[i])-1)^2*(1+10sin(π*y(x[i])+1)^2)
        sum += new
    end

    return term1 + sum + term3
end

levy3 (generic function with 1 method)

## 1 Exhaustive Search

In [14]:
"""
Performs an exhaustive search for a function using a discretized search space.

The total number of function calls will be d^n, where d is the number of
dimensions, and n is the length of the discretized search space.

    Args:
      fn (Function): Objective function
      x_range (Array): Range of x values to explore
      dimensions (Int64): Dimensionality of search space
      spacing (Float64, optional): Discretization
      verbose (Bool, optional): Print results

    Returns:
      Tuple: (x, fx, f_calls) where x is the minimizer, fx is the minimum, and
      f_calls is the number of function calls made.
"""
function exhaustive_search(fn::Function, x_range::Array, dimensions::Int64;
                           spacing=.1, verbose=true)
    f_calls = 1

    i = 1
    k = [1 for _ in 1:dimensions]
    min_x, max_x = x_range
    x_values = collect(min_x:spacing:max_x)
    n_x_values = length(x_values)
    x = [x_values[begin] for _ in 1:dimensions]
    x_star = copy(x)
    f_star = fn(x_star)
    n = length(x)

    while i <= n
        k[i] += 1

        if k[i] > n_x_values
            x[i] = min_x
            k[i] = 1
            i += 1
            continue
        end

        x[i] = x_values[k[i]]

        fnew = fn(x)
        f_calls += 1

        if verbose && (f_calls % 1000000 == 0)
            println("Iterations: $(f_calls)\nx*: $(x_star)\nf*: $(f_star)\n")
        end

        if fnew < f_star
            f_star = fnew
            x_star = copy(x)
        end

        i = 1
    end

    if verbose
        println("Iterations: $(f_calls)\nx*: $(x_star)\nf*: $(f_star)\n")
    end

    return x_star, f_star, f_calls
end

exhaustive_search

In [15]:
(x, fx, f_calls) = exhaustive_search(levy3, [-2, 6], 4; verbose=true)

Iterations: 1000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 2000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 3000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 4000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 5000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 6000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 7000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 8000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 9000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 10000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 11000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 12000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 13000000
x*: [1.0, 1.0, 1.0, -2.0]
f*: 1.4997597826618576e-32

Iterations: 14000000
x*: [1.0, 1.0

([1.0, 1.0, 1.0, -2.0], 1.4997597826618576e-32, 43046721)

## 2 Random Hill Climbing

In [16]:
"""
Performs a random hill climb solution for a discretized search space.

    Args:
      fn (Function): Objective function
      x0 (Array{Any}): Starting point
      h (Float64, optional): Discretization
      max_failed (Int64, optional): Max number of failed moves before exiting.
        Defaults to the dimensionality of the design space.
      verbose (Bool, optional): Optionally print results

    Returns:
      tuple: (x, fx, f_calls) where x is the minimizer, fx is the minimum, and
      f_calls is the number of function calls made.
"""
function random_hill(fn::Function, x0::Array{Float64};
                     h=.1, max_failed=nothing, verbose=false)
    x = copy(x0)
    fx = fn(x)
    n = length(x)
    failed = 0
    f_calls = 1

    # use an I matrix to randomly determine direction
    e_hat = Matrix(1I, n, n)

    if isnothing(max_failed)
        max_failed = n
    end

    while failed < max_failed
        d = e_hat[:, rand(1:n)] # randomly choose basis vec
        d *= rand([-1, 1]) # randomly choose pos/neg
        
        xnew = x + h*d
        fnew = fn(xnew)
        f_calls += 1

        if fnew < fx
            fx = fnew
            x = xnew
            failed = 0
        else
            failed += 1
        end
    end

    if verbose
        println("Iterations: $(f_calls)\nx*: $(x)\nf*: $(fx)\n")
    end

    return x, fx, f_calls
end

random_hill

In [17]:
x0 = [6.0, -2.0, -2.0, 6.0]
random_hill(levy3, x0; max_failed=100, verbose=true)
random_hill(levy3, x0; h=.01, max_failed=100, verbose=true)

Iterations: 298
x*: [3.800000000000007, -0.09999999999999937, -0.09999999999999937, 4.900000000000004]
f*: 4.388225844856102

Iterations: 1856
x*: [3.790000000000047, -0.08999999999999836, -0.08999999999999836, 4.940000000000023]
f*: 4.376401911442446



([3.790000000000047, -0.08999999999999836, -0.08999999999999836, 4.940000000000023], 4.376401911442446, 1856)

In [18]:
x0 = [4.2, -1.2, 2.4, -3.8]
random_hill(levy3, x0; max_failed=100, verbose=true)
random_hill(levy3, x0; h=.01, max_failed=100, verbose=true)

Iterations: 248
x*: [3.8000000000000007, -0.10000000000000003, 0.9999999999999988, -1.9999999999999982]
f*: 1.25024294018724

Iterations: 1796
x*: [3.790000000000009, -0.08999999999999903, 1.0000000000000075, -2.000000000000038]
f*: 1.2499870520527887



([3.790000000000009, -0.08999999999999903, 1.0000000000000075, -2.000000000000038], 1.2499870520527887, 1796)

In [19]:
x0 = [0.0, 0.0, 0.0, 3.2]
random_hill(levy3, x0; max_failed=100, verbose=true)
random_hill(levy3, x0; h=.01, max_failed=100, verbose=true)

Iterations: 161
x*: [0.9999999999999999, -0.1, -0.1, 2.9]
f*: 1.7164262632557858

Iterations: 925
x*: [1.0000000000000007, -0.09, -0.09, 2.920000000000006]
f*: 1.7157256258683309



([1.0000000000000007, -0.09, -0.09, 2.920000000000006], 1.7157256258683309, 925)

In [20]:
x0 = [1.0, 1.0, 1.0, 1.0]
random_hill(levy3, x0; max_failed=100, verbose=true)
random_hill(levy3, x0; h=.01, max_failed=100, verbose=true)

Iterations: 107
x*: [1.0, 1.0, 1.0, 0.8]
f*: 0.536790836378138

Iterations: 253
x*: [1.0, 1.0, 1.0, 0.8399999999999999]
f*: 0.5352769012949443



([1.0, 1.0, 1.0, 0.8399999999999999], 0.5352769012949443, 253)

## 3 Simulated Annealing

In [21]:
max_iter = 1000
SA_options = Optim.Options(iterations=max_iter)

                x_abstol = 0.0
                x_reltol = 0.0
                f_abstol = 0.0
                f_reltol = 0.0
                g_abstol = 1.0e-8
                g_reltol = 1.0e-8
          outer_x_abstol = 0.0
          outer_x_reltol = 0.0
          outer_f_abstol = 0.0
          outer_f_reltol = 0.0
          outer_g_abstol = 1.0e-8
          outer_g_reltol = 1.0e-8
           f_calls_limit = 0
           g_calls_limit = 0
           h_calls_limit = 0
       allow_f_increases = true
 allow_outer_f_increases = true
        successive_f_tol = 1
              iterations = 1000
        outer_iterations = 1000
             store_trace = false
           trace_simplex = false
              show_trace = false
          extended_trace = false
              show_every = 1
                callback = nothing
              time_limit = NaN


In [22]:
x0 = [6.0, -2.0, -2.0, 6.0]
res = optimize(levy3, x0, SimulatedAnnealing(), SA_options)
display(res)
println("x*: $(res.minimizer)")
println("f*: $(res.minimum)")

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     7.639543e-02

 * Found with
    Algorithm:     Simulated Annealing

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    1001


x*: [0.8003069318499135, 1.0182522443060409, 0.889320246399253, -2.594099091581852]
f*: 0.07639542781377873


In [23]:
x0 = [4.2, -1.2, 2.4, -3.8]
res = optimize(levy3, x0, SimulatedAnnealing(), SA_options)
display(res)
println("x*: $(res.minimizer)")
println("f*: $(res.minimum)")

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     7.736743e-02

 * Found with
    Algorithm:     Simulated Annealing

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    1001


x*: [1.0103744043813154, 1.0793328573990968, 0.4745316273867294, -2.173018410673434]
f*: 0.07736743194470062


In [24]:
x0 = [0.0, 0.0, 0.0, 3.2]
res = optimize(levy3, x0, SimulatedAnnealing(), SA_options)
display(res)
println("x*: $(res.minimizer)")
println("f*: $(res.minimum)")

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     4.072622e-02

 * Found with
    Algorithm:     Simulated Annealing

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    1001


x*: [0.9297696827614319, 0.9508275997821172, 1.1098427525347336, -1.4405250132061074]
f*: 0.04072622069601746


In [25]:
x0 = [1.0, 1.0, 1.0, 1.0]
res = optimize(levy3, x0, SimulatedAnnealing(), SA_options)
display(res)
println("x*: $(res.minimizer)")
println("f*: $(res.minimum)")

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     2.850035e-02

 * Found with
    Algorithm:     Simulated Annealing

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    1001


x*: [1.1486386466452287, 1.0637634948351424, 0.9851677376667263, -1.9683423356611223]
f*: 0.028500348833225532


## 4 Particle Swarm

In [26]:
max_iter = 1000
PS_options = Optim.Options(iterations=max_iter)

                x_abstol = 0.0
                x_reltol = 0.0
                f_abstol = 0.0
                f_reltol = 0.0
                g_abstol = 1.0e-8
                g_reltol = 1.0e-8
          outer_x_abstol = 0.0
          outer_x_reltol = 0.0
          outer_f_abstol = 0.0
          outer_f_reltol = 0.0
          outer_g_abstol = 1.0e-8
          outer_g_reltol = 1.0e-8
           f_calls_limit = 0
           g_calls_limit = 0
           h_calls_limit = 0
       allow_f_increases = true
 allow_outer_f_increases = true
        successive_f_tol = 1
              iterations = 1000
        outer_iterations = 1000
             store_trace = false
           trace_simplex = false
              show_trace = false
          extended_trace = false
              show_every = 1
                callback = nothing
              time_limit = NaN


In [27]:
x0 = [6.0, -2.0, -2.0, 6.0]
res = optimize(levy3, x0, ParticleSwarm(), PS_options)
display(res)
println("x*: $(res.minimizer)")
println("f*: $(res.minimum)")

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     1.654089e-15

 * Found with
    Algorithm:     Particle Swarm

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    5004
    ∇f(x) calls:   0


x*: [0.9999999999500522, 1.0000000032092258, 0.9999999428824307, -2.000000003117009]
f*: 1.6540888649904657e-15


In [28]:
x0 = [4.2, -1.2, 2.4, -3.8]
res = optimize(levy3, x0, ParticleSwarm(), PS_options)
display(res)
println("x*: $(res.minimizer)")
println("f*: $(res.minimum)")

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     3.314326e-19

 * Found with
    Algorithm:     Particle Swarm

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    5004
    ∇f(x) calls:   0


x*: [0.9999999995664907, 1.000000000056626, 1.0000000004826368, -2.0000000001031575]
f*: 3.314326024542856e-19


In [29]:
x0 = [0.0, 0.0, 0.0, 3.2]
res = optimize(levy3, x0, ParticleSwarm(), PS_options)
display(res)
println("x*: $(res.minimizer)")
println("f*: $(res.minimum)")

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     8.952825e-02

 * Found with
    Algorithm:     Particle Swarm

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    5004
    ∇f(x) calls:   0


x*: [1.0000000033023606, 1.000000000963575, -0.09299393943576975, -1.9999996758729255]
f*: 0.08952825041646703


In [30]:
x0 = [1.0, 1.0, 1.0, 1.0]
res = optimize(levy3, x0, ParticleSwarm(), PS_options)
display(res)
println("x*: $(res.minimizer)")
println("f*: $(res.minimum)")

 * Status: failure (reached maximum number of iterations)

 * Candidate solution
    Final objective value:     1.313701e-26

 * Found with
    Algorithm:     Particle Swarm

 * Convergence measures
    |x - x'|               = NaN ≰ 0.0e+00
    |x - x'|/|x'|          = NaN ≰ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = NaN ≰ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1000
    f(x) calls:    5004
    ∇f(x) calls:   0


x*: [0.9999999999999999, 1.0, 1.0, -2.000000000000324]
f*: 1.313701425985649e-26


## 5 Analysis

**Discuss differences in the four approaches above. Consider observations based on the robustness, generality, accuracy, ease of use, and efficiency of the methods.**

**Robustness:**

In general, the stochastic methods utilized in this notebook are robust in the sense that they can typically find at least a local minimum within the search space. However, for the case described, some clearly had more trouble than others in finding the global minimum, particularly Random Hill Climbing. This is expected, as this method makes no particular efforts to escape falling into local minima.

**Generality:**

All the methods utilized are generally applicable to discretized optimization problems, though for a multimodal objective function, Random Hill climbing stands out as a poor fit. Conversely, if this problem had been unimodal, Simulated Annealing and Particle Swarm may have proved to be overkill. Exhaustive Search is arguably the most applicable, though only for problems that are not intractably large.

**Accuracy:**

As expected, Exhaustive Search is the most accurate method, as it completely trades off performance for accuracy. Among the other methods, PSO performed the best, often (but not always) achieving the global optimum. Simulated Annealing had some difficulties with local minima, and Random Hill Climbing had significant difficulties.

**Ease of use:**

Exhaustive Search is particularly easy to use, though its performance would become prohibitive for a larger search space. Random Hill climbing is also easy to use, with fairly intuitive knobs for discretization and convergence that can easily be experimented with and iterated upon to achieve better results. Simulated Annealing and Particle Swarm are relatively more complicated; though their analogies to physical systems are useful for conceptual understanding, the effects of particular knobs like Temperature and Population on results is less clear.

**Efficiency:**

While all of the methods described require a significant number of function calls, Exhaustive Search is, as expected, the least efficient, as it searches the entire search space, even if it happens to find the optimal result along the way. Simulated Annealing (SA) and Particle Swarm (PSO) also require many function calls: SA is often exploring to escape local minima; and PSO is informed via function calls for all particles in the population. Random hill climbing is presumably the most efficient, though it trades off accuracy/precision, as random steps can lead to very different outcomes depending on their directions.